<a href="https://colab.research.google.com/github/Jhonyd55/MichiRazaDetector/blob/main/MichiRazaDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title instalar complementos si no estan instalados
import importlib.util
import subprocess
import sys

def install_package(package_name):
    if importlib.util.find_spec(package_name) is None:
      print(f"🔧 Instalando {package_name}...")
      subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
      print(f"{package_name} instalado con éxito.")
    else:
      print(f"{package_name} ya está instalado.")

#package_name = "google_images_download"
package_name = "bing-image-downloader"
install_package(package_name)



In [ ]:
import shutil
import os

folder = 'dataset'

if os.path.exists(folder):
    shutil.rmtree(folder)
    print(f"✅ Carpeta '{folder}' eliminada correctamente.")
else:
    print(f"⚠️ La carpeta '{folder}' no existe.")


In [ ]:
# @title Razas de gatos segun la TICA

#search_queries = ['Abyssinian cat', 'Bengal cat', 'Persian cat']

search_queries = [
    "Abyssinian cat",
    "American Bobtail cat",
    "American Curl cat",
    "American Shorthair cat",
    "American Wirehair cat",
    "Balinese cat",
    "Bengal cat",
    "Birman cat",
    "Bombay cat",
    "British Longhair cat",
    "British Shorthair cat",
    "Burmese cat",
    "Burmilla cat",
    "Chartreux cat",
    "Colorpoint Shorthair cat",
    "Cornish Rex cat",
    "Cymric cat",
    "Devon Rex cat",
    "Egyptian Mau cat",
    "European Burmese cat",
    "Exotic Shorthair cat",
    "Havana Brown cat",
    "Himalayan cat",
    "Japanese Bobtail cat",
    "Khao Manee cat",
    "Korat cat",
    "LaPerm cat",
    "Lykoi cat",
    "Maine Coon cat",
    "Manx cat",
    "Norwegian Forest cat",
    "Ocicat cat",
    "Oriental cat",
    "Persian cat",
    "Pixiebob cat",
    "Ragamuffin cat",
    "Ragdoll cat",
    "Russian Blue cat",
    "Savannah cat",
    "Scottish Fold cat",
    "Selkirk Rex cat",
    "Siamese cat",
    "Siberian cat",
    "Singapura cat",
    "Snowshoe cat",
    "Somali cat",
    "Sphynx cat",
    "Tonkinese cat",
    "Toyger cat",
    "Turkish Angora cat",
    "Turkish Van cat"
]


In [ ]:
license_filters = [
    "license-cc_publicdomain", # Dominio Público
    "license-cc_attribution", # Atribución (CC BY)
    "license-cc_sharealike", # Compartir Igual (CC BY-SA)
    "license-cc_noncommercial", # No Comercial (CC BY-NC)
    "license-cc_noncommercial_sharealike", # No Comercial - Compartir Igual (CC BY-NC-SA)
    "license-cc_noncommercial_noderivatives", # No Comercial - Sin Obras Derivadas (CC BY-NC-ND)
    "license-cc_noderivatives" # Sin Obras Derivadas (CC BY-ND)
]

In [ ]:
import os
import time
from bing_image_downloader import downloader

def download_filtered_images(search_queries, license_filters, base_dir="dataset", limit=100, timeout_per_license=300): # tiempo de espera por licencia en segundos (5 minutos)
    for query in search_queries:
        query_dir = os.path.join(base_dir, query)#.replace(" ", "_"))
        downloaded_count = 0

        print(f"⬇️ Descargando imágenes de: {query} en carpeta {query_dir}")
        # Verificar si la carpeta ya existe y tiene suficientes imágenes
        if os.path.exists(query_dir):
            downloaded_count = len([f for f in os.listdir(query_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            print(f"⬇️ Cantidad de imágenes: {downloaded_count} en carpeta")
            if downloaded_count >= limit:
                print(f"✅ Ya existe {query_dir} con {downloaded_count} imágenes, omitiendo descarga.")
                continue
        else:
              #downloaded_count = len([f for f in os.listdir(query_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
              #print(f"Continuando la descarga para {query_dir}, {downloaded_count} imágenes ya existen.")
              ###
              # Iterar a través de los filtros de licencia para la consulta actual
              for license_filter in license_filters:
                print(f"  Probando filtro de licencia: {license_filter}")
                start_time = time.time()
                initial_count = downloaded_count # Mantener un registro de las imágenes antes de este intento de licencia

                try:
                    downloader.download(
                        query,
                        limit=limit - downloaded_count, # Descargar imágenes restantes
                        output_dir=base_dir,
                        adult_filter_off=True,
                        force_replace=False,
                        timeout=60, # Tiempo de espera para la descarga de imágenes individuales
                        filter=license_filter,
                        verbose=False # Reducir la verbosidad para evitar demasiada salida
                    )
                except Exception as e:
                    print(f"  Error durante la descarga con la licencia {license_filter}: {e}")

                # 🔍 Eliminar archivos que no sean .jpg, .png, o .jpeg en el directorio de la consulta
                if os.path.exists(query_dir):
                    for fname in os.listdir(query_dir):
                        if not (fname.lower().endswith(".jpg") or fname.lower().endswith(".png") or fname.lower().endswith(".jpeg")):
                            try:
                                os.remove(os.path.join(query_dir, fname))
                                print(f"🗑️ Eliminado archivo no compatible: {fname} en {query_dir}")
                            except Exception as e:
                                print(f"❌ Error al eliminar el archivo {fname}: {e}")


                # Actualizar el número de imágenes descargadas después del intento de descarga
                if os.path.exists(query_dir):
                    downloaded_count = len([f for f in os.listdir(query_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

                print(f"⬇️ Cantidad de imágenes después de la limpieza: {downloaded_count} en carpeta")
                # Verificar si se han descargado suficientes imágenes
                if downloaded_count >= limit:
                    print(f"✅ Descargadas {downloaded_count} imágenes para {query}. Pasando a la siguiente consulta.")
                    break # Pasar a la siguiente consulta si se alcanza el límite

                # Verificar si se ha alcanzado el tiempo de espera para esta licencia y no se han descargado nuevas imágenes
                elapsed_time = time.time() - start_time
                if elapsed_time >= timeout_per_license and downloaded_count == initial_count:
                    print(f"  Tiempo de espera ({timeout_per_license}s) alcanzado para la licencia {license_filter} sin nuevas imágenes. Cambiando a la siguiente licencia.")
                    # Continuar con el siguiente filtro de licencia


        # Después de intentar todas las licencias, verificar si se alcanzó el límite.
        if downloaded_count < limit:
            print(f"⚠️ No se pudieron descargar {limit} imágenes para {query} después de probar todas las licencias. Descargadas {downloaded_count}.")
                 ###










# Ahora llama a la función con la lista de consultas de búsqueda y filtros de licencia
# Asegúrate de que la lista license_filters esté definida en una celda anterior

# Define los filtros de licencia para ciclar
# license_filters = [
#     "license-cc_publicdomain",
#     "license-cc_attribution",
#     "license-cc_sharealike",
#     "license-cc_noncommercial",
#     "license-cc_noncommercial_sharealike",
#     "license-cc_noncommercial_noderivatives",
#     "license-cc_noderivatives"
# ]

download_filtered_images(search_queries, license_filters, limit=100, timeout_per_license=300) # 5 minutos de tiempo de espera por licencia

In [ ]:
# @title Imagenes antes de la Selección
analysis_df = analyze_dataset_structure('dataset')
if analysis_df is not None:
  display(analysis_df)

In [ ]:
# @title Herramienta para eliminar las imagenes que no sirven para el entrenamiento

import os
import glob
from IPython.display import display, clear_output
import ipywidgets as widgets
from PIL import Image as PILImage
from io import BytesIO

# Carpeta base del dataset
dataset_path = 'dataset'

# Dictionary to store selected images for deletion
selected_images = {}

# Obtener las razas disponibles
def get_race_folders(base_path):
    return [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]

# Mostrar imágenes con botón de eliminar y checkbox de selección
def show_images_grid(race):
    clear_output(wait=True)
    folder_path = os.path.join(dataset_path, race)
    image_paths = glob.glob(os.path.join(folder_path, '*'))

    if not image_paths:
        print(f"⚠️ No hay imágenes en {race}")
        return

    print(f"🖼️ Selecciona las imágenes a eliminar y haz clic en 🗑️ para eliminarlas. Raza: {race}")

    items = []
    # Clear previous selections when changing race
    selected_images.clear()

    for path in image_paths:
        try:
            # Cargar imagen
            with open(path, 'rb') as f:
                img = PILImage.open(f).convert("RGB")
                img.thumbnail((100, 100))
                buffer = BytesIO()
                img.save(buffer, format='JPEG')
                img_data = buffer.getvalue()

            # Widget de imagen
            image_widget = widgets.Image(
                value=img_data,
                format='jpg',
                width=220,
                height=220
            )

            # Checkbox para seleccionar
            select_checkbox = widgets.Checkbox(
                value=False,
                description='Seleccionar',
                indent=False,
                layout=widgets.Layout(width='auto')
            )

            # Associate checkbox with image path
            selected_images[path] = select_checkbox

            # Botón para eliminar (will be modified later to delete selected)
            # For now, keep the single delete button functionality as a placeholder
            delete_button = widgets.Button(
                description='🗑️',
                layout=widgets.Layout(width='220px'),
                tooltip=path,
                button_style='danger'
            )

            def on_click(b, path=path):
                try:
                    os.remove(path)
                    print(f"✅ Imagen eliminada: {os.path.basename(path)}")
                    show_images_grid(race)  # Recargar la grilla
                except Exception as e:
                    print(f"❌ Error: {e}")

            # delete_button.on_click(on_click) # Temporarily disable single delete

            # Agrupar imagen + checkbox + button
            box = widgets.VBox([image_widget, select_checkbox, delete_button])
            items.append(box)

        except:
            continue

    # Mostrar en grilla
    grid = widgets.GridBox(
        children=items,
        layout=widgets.Layout(grid_template_columns="repeat(3, 220px)")
    )

    # Add a master delete button for selected images (will be implemented in the next subtask)
    master_delete_button = widgets.Button(
        description='Eliminar Seleccionadas 🗑️',
        button_style='danger',
        layout=widgets.Layout(width='auto')
    )

    # display(race_dropdown, grid) # Modify display to include master delete button
    display(race_dropdown, master_delete_button, grid)


# Dropdown de raza
race_dropdown = widgets.Dropdown(
    options=get_race_folders(dataset_path),
    description='Raza:',
)
race_dropdown.observe(lambda change: show_images_grid(change['new']), names='value')

# Mostrar inicial
display(race_dropdown)
show_images_grid(race_dropdown.value)

In [ ]:
# @title preprocesamiento
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image

# 📍 Ruta original y de salida
ORIG_DIR = 'dataset'
DEST_DIR = 'processed_dataset'
IMG_SIZE = (224, 224)

# 🧽 Limpiar carpeta de salida si ya existe
if os.path.exists(DEST_DIR):
    shutil.rmtree(DEST_DIR)
os.makedirs(DEST_DIR, exist_ok=True)

# 🔁 Función para preprocesar una imagen
def preprocess_and_save(src_path, dest_path):
    try:
        img = Image.open(src_path).convert('RGB')
        img = img.resize(IMG_SIZE)
        img.save(dest_path)
    except Exception as e:
        print(f"❌ Error con {src_path}: {e}")

# 🔀 División y preprocesamiento
for class_name in os.listdir(ORIG_DIR):
    class_path = os.path.join(ORIG_DIR, class_name)
    if not os.path.isdir(class_path): continue

    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    train_files, temp_files = train_test_split(images, train_size=0.7, random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)  # 15/15

    split_data = {
        'train': train_files,
        'val': val_files,
        'test': test_files
    }

    for split, files in split_data.items():
        out_dir = os.path.join(DEST_DIR, split, class_name)
        os.makedirs(out_dir, exist_ok=True)

        for file in files:
            src = os.path.join(class_path, file)
            dst = os.path.join(out_dir, file)
            preprocess_and_save(src, dst)

print("✅ Preprocesamiento completo y dataset dividido.")


In [ ]:

# @title 🧠 Red Preentrenada **MobileNetV2**
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Tamaño de entrada y batch
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 100

# Ruta del dataset procesado
DATASET_DIR = 'processed_dataset'

# Cargar datos con generadores
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(DATASET_DIR, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    os.path.join(DATASET_DIR, 'val'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    os.path.join(DATASET_DIR, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Número de clases
num_classes = train_gen.num_classes

# Base preentrenada MobileNetV2 sin las capas superiores
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar capas del modelo base
base_model.trainable = False

# Añadir capas finales
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS
)


In [ ]:
# @title 📊 Evaluar el modelo
loss, acc = model.evaluate(test_gen)
print(f"📈 Precisión en test: {acc:.2%}")

In [ ]:
# prompt: generar un algoritmo  para evaluar la matriz de confusion y la metrica con una buena visualizcion del proyecto

from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Obtener predicciones del conjunto de prueba
test_pred_raw = model.predict(test_gen)
test_pred = np.argmax(test_pred_raw, axis=1)

# Obtener etiquetas verdaderas del conjunto de prueba
test_true_labels = test_gen.classes

# Obtener los nombres de las clases en el mismo orden que el generador
class_names = list(test_gen.class_indices.keys())

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(test_true_labels, test_pred)

# Visualizar la matriz de confusión
plt.figure(figsize=(20, 15))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Clase Predicha')
plt.ylabel('Clase Verdadera')
plt.title('Matriz de Confusión')
plt.show()

# Generar el reporte de clasificación
print("\n📊 Reporte de Clasificación:")
print(classification_report(test_true_labels, test_pred, target_names=class_names))

# Visualizar la precisión y pérdida durante el entrenamiento
plt.figure(figsize=(12, 5))

# Precisión
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Precisión Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión Validación')
plt.title('Precisión del Modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend()

# Pérdida
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Pérdida Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida Validación')
plt.title('Pérdida del Modelo')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# 1. Obtener predicciones y etiquetas reales del conjunto de prueba
test_gen.reset() # Reiniciar el generador para asegurar el orden
y_pred_probs = model.predict(test_gen)
y_true = test_gen.classes # Obtener las etiquetas verdaderas

# Binarizar las etiquetas verdaderas para cada clase
y_true_binarized = label_binarize(y_true, classes=np.arange(num_classes))

# 2. Calcular la curva ROC y el AUC para cada clase
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_binarized[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# 3. Graficar las curvas ROC en grupos de 10
classes_per_plot = 10
num_plots = (num_classes + classes_per_plot - 1) // classes_per_plot

class_names = list(test_gen.class_indices.keys())

for plot_idx in range(num_plots):
    plt.figure(figsize=(10, 8))
    start_idx = plot_idx * classes_per_plot
    end_idx = min((plot_idx + 1) * classes_per_plot, num_classes)

    for i in range(start_idx, end_idx):
        plt.plot(fpr[i], tpr[i], lw=2,
                 label='Curva ROC de la clase {0} (AUC = {1:0.2f})'
                 ''.format(class_names[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Tasa de Falsos Positivos (FPR)')
    plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
    plt.title(f'Curva ROC y AUC para clases {start_idx+1} a {end_idx}')
    plt.legend(loc="lower right")
    plt.show()

# Calcular AUC macro y micro promedio (opcional) - Esta parte permanece igual
all_y_true = y_true_binarized.ravel()
all_y_pred_probs = y_pred_probs.ravel()

# Macro promedio
# Necesita manejar el caso multi-clase para el promedio macro
# fpr_macro, tpr_macro, _ = roc_curve(all_y_true, all_y_pred_probs)
# roc_auc_macro = auc(fpr_macro, tpr_macro)

# print(f"\nAUC Promedio Macro: {roc_auc_macro:.2f}")

# Micro promedio (más apropiado para multi-clase)
fpr_micro, tpr_micro, _ = roc_curve(y_true_binarized.ravel(), y_pred_probs.ravel())
roc_auc_micro = auc(fpr_micro, tpr_micro)

print(f"\nAUC Promedio Micro: {roc_auc_micro:.2f}")

In [ ]:
# @title Busca una imagen random de validación y la evalúa
import random
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

# Función para predecir una imagen aleatoria de validación
def predict_random_image_from_validation():
    val_dir = os.path.join(DATASET_DIR, 'val')
    class_names = os.listdir(val_dir)

    # Elegir clase y archivo aleatorio
    true_class = random.choice(class_names)
    img_file = random.choice(os.listdir(os.path.join(val_dir, true_class)))
    img_path = os.path.join(val_dir, true_class, img_file)

    # Cargar imagen
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalizar
    img_array = np.expand_dims(img_array, axis=0)  # Lote de 1

    # Predicción
    preds = model.predict(img_array)[0]
    top_idx = np.argmax(preds)
    pred_class = train_gen.class_indices
    label_map = {v: k for k, v in pred_class.items()}
    predicted_class = label_map[top_idx]
    confidence = preds[top_idx]

    # Mostrar resultado
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"✅ Real: {true_class}\n🔍 Predicho: {predicted_class} ({confidence:.2%})")
    plt.show()

In [ ]:
predict_random_image_from_validation()


In [ ]:

package_name = "gradio"
install_package(package_name)

In [ ]:
import gradio as gr

# Diccionario para mapear índice → nombre de clase
idx_to_class = {v: k for k, v in train_gen.class_indices.items()}

# Función de predicción para Gradio
def predict_image(img):
    from tensorflow.keras.preprocessing import image as keras_image

    img = img.resize(IMG_SIZE)
    img_array = keras_image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)[0]

    # Crear diccionario {raza: probabilidad}
    result = {idx_to_class[i]: float(f"{p:.4f}") for i, p in enumerate(predictions)}

    return result

# Crear interfaz
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="🐱 Michi Raza Detector",
    description="Sube una imagen de un gato para identificar su raza. Muestra las razas más probables."
)

# Ejecutar interfaz
iface.launch(debug=True)

In [ ]:
# @title Para Guardar y descargar el modelo entrenado

model.save("michi_model.h5")



In [ ]:
# @title Para convertirlo en una version para usar en un proyecto de android
import tensorflow as tf

# Cargar el modelo .h5
loaded_model = tf.keras.models.load_model("michi_model.h5")

# Convertir a TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Guardar el archivo .tflite
with open("michi_model.tflite", "wb") as f:
    f.write(tflite_model)


In [ ]:

# @title Descargar el **modelo**
from google.colab import files
files.download("michi_model.tflite")


In [ ]:
# @title descargar dataset
import shutil

# Comprime la carpeta dataset en un archivo ZIP
shutil.make_archive("dataset_gatitos", 'zip', "dataset")

